In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout, TimeDistributed
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-03-21 22:35:35.928394: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-21 22:35:35.931553: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-21 22:35:35.939246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742585735.952658 1286605 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742585735.956377 1286605 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 22:35:35.971670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=False))

    model.add(Dropout(0.2))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [ ]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,0.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,0,0.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0
2,0,0.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-03-21 22:35:37.881843: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 144, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc (LTC)                       │ (None, 144, 10)        │           828 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 144, 10)        │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_2 (LTC)                     │ (None, 10)             │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,641 (10.32 KB)

 Trainable params: 2,641 (10.32 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 868ms/step - accuracy: 0.5471 - auc: 0.6564 - loss: 0.6912 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6962
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 557ms/step - accuracy: 0.4140 - auc: 0.5427 - loss: 0.7064 - val_accuracy: 0.5000 - val_auc: 0.3667 - val_loss: 0.6941
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 552ms/step - accuracy: 0.4948 - auc: 0.5051 - loss: 0.6949 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 574ms/step - accuracy: 0.4587 - auc: 0.4411 - loss: 0.7079 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 562ms/step - accuracy: 0.5136 - auc: 0.4545 - loss: 0.7036 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 559ms/step - accuracy: 0.4605 - auc: 0.3779 - loss: 0.7127 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6933
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 557ms

1it [14:01, 841.76s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 800ms/step - accuracy: 0.5780 - auc: 0.6075 - loss: 0.6790 - val_accuracy: 0.5000 - val_auc: 0.7000 - val_loss: 0.6943
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 602ms/step - accuracy: 0.5851 - auc: 0.5937 - loss: 0.6748 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6884
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 607ms/step - accuracy: 0.5572 - auc: 0.5939 - loss: 0.6756 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 598ms/step - accuracy: 0.3924 - auc: 0.3419 - loss: 0.7104 - val_accuracy: 0.5000 - val_auc: 0.4667 - val_loss: 0.6933
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 597ms/step - accuracy: 0.5426 - auc: 0.5377 - loss: 0.6875 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6888
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 604ms/step - accuracy: 0.4859 - auc: 0.4944 - loss: 0.6862 - val_accuracy: 0.6000 - val_auc: 0.6000 - val_loss: 0.6886
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 601ms

2it [28:13, 847.70s/it]

...

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 804ms/step - accuracy: 0.4821 - auc: 0.4758 - loss: 0.7134 - val_accuracy: 0.5000 - val_auc: 0.7000 - val_loss: 0.6916
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 597ms/step - accuracy: 0.4059 - auc: 0.3305 - loss: 0.7321 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6929
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 608ms/step - accuracy: 0.5498 - auc: 0.6077 - loss: 0.6812 - val_accuracy: 0.5000 - val_auc: 0.6667 - val_loss: 0.6924
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 610ms/step - accuracy: 0.5435 - auc: 0.5272 - loss: 0.6913 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6889
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 608ms/step - accuracy: 0.4897 - auc: 0.5145 - loss: 0.6928 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6875
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 608ms/step - accuracy: 0.5985 - auc: 0.6065 - loss: 0.6818 - val_accuracy: 0.6333 - val_auc: 0.6333 - val_loss: 0.6850
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 611ms

4it [56:39, 851.85s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 808ms/step - accuracy: 0.6272 - auc: 0.4749 - loss: 0.6973 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 606ms/step - accuracy: 0.5125 - auc: 0.5726 - loss: 0.6782 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 602ms/step - accuracy: 0.5937 - auc: 0.5002 - loss: 0.6926 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 603ms/step - accuracy: 0.4009 - auc: 0.4991 - loss: 0.6939 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 608ms/step - accuracy: 0.4810 - auc: 0.5930 - loss: 0.6949 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 597ms/step - accuracy: 0.4652 - auc: 0.4645 - loss: 0.6968 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 612ms

5it [1:12:33, 870.75s/it]

CPU times: user 2h 50min 28s, sys: 1h 24min 24s, total: 4h 14min 53s
Wall time: 1h 12min 33s


In [ ]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

Epoch 1: TRAIN Accuracy = 0.517 Loss = 0.701 AUC = 0.541
Epoch 1: VAL Accuracy = 0.5 Loss = 0.696 AUC = 0.633
Epoch 2: TRAIN Accuracy = 0.469 Loss = 0.702 AUC = 0.49
Epoch 2: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.547
Epoch 3: TRAIN Accuracy = 0.52 Loss = 0.692 AUC = 0.539
Epoch 3: VAL Accuracy = 0.5 Loss = 0.693 AUC = 0.56
Epoch 4: TRAIN Accuracy = 0.486 Loss = 0.695 AUC = 0.472
Epoch 4: VAL Accuracy = 0.5 Loss = 0.693 AUC = 0.52
Epoch 5: TRAIN Accuracy = 0.503 Loss = 0.696 AUC = 0.514
Epoch 5: VAL Accuracy = 0.52 Loss = 0.691 AUC = 0.573
Epoch 6: TRAIN Accuracy = 0.486 Loss = 0.698 AUC = 0.472
Epoch 6: VAL Accuracy = 0.547 Loss = 0.691 AUC = 0.573
Epoch 7: TRAIN Accuracy = 0.529 Loss = 0.692 AUC = 0.54
Epoch 7: VAL Accuracy = 0.547 Loss = 0.69 AUC = 0.58
Epoch 8: TRAIN Accuracy = 0.523 Loss = 0.692 AUC = 0.526
Epoch 8: VAL Accuracy = 0.547 Loss = 0.688 AUC = 0.613
Epoch 9: TRAIN Accuracy = 0.554 Loss = 0.687 AUC = 0.525
Epoch 9: VAL Accuracy = 0.553 Loss = 0.686 AUC = 0.6
Epoch 10: